In [1]:
!pip install capstone -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.5 MB/s eta 0:00:00


In [2]:
import struct
from capstone import *

In [3]:
# Create fake binary
path = "/kaggle/working/rev_example.bin"

with open(path, "wb") as f:
    f.write(b"MAGC")                      # Magic header
    f.write(struct.pack("<I", 9999))     # Integer
    f.write(b"hidden_password")          # Hidden string
    f.write(b"\x55\x48\x89\xe5\x90\x5d\xc3")  # Machine code

print("Binary file created:", path)

Binary file created: /kaggle/working/rev_example.bin


In [4]:
!cat /kaggle/working/rev_example.bin

MAGC'  hidden_passwordUH��]�

In [5]:
# Read raw binary

with open(path, "rb") as f:
    data = f.read()

print("Size:", len(data), "bytes")
print("Hex dump:", data.hex())

Size: 30 bytes
Hex dump: 4d4147430f27000068696464656e5f70617373776f7264554889e5905dc3


In [6]:
# Parse binary structure

magic = data[:4]
number = struct.unpack("<I", data[4:8])[0]
hidden = data[8:-7]
machine_code = data[-7:]

print("Magic:", magic)
print("Number:", number)
print("Hidden string:", hidden.decode())

Magic: b'MAGC'
Number: 9999
Hidden string: hidden_password


In [7]:
# Extract printable strings

strings = []
current = ""

for byte in data:
    if 32 <= byte <= 126:
        current += chr(byte)
    else:
        if len(current) >= 4:
            strings.append(current)
        current = ""

if len(current) >= 4:
    strings.append(current)

print("Found strings:")
for s in strings:
    print("-", s)

Found strings:
- MAGC
- hidden_passwordUH


In [8]:
# Disassemble machine code

md = Cs(CS_ARCH_X86, CS_MODE_64)

for i in md.disasm(machine_code, 0):
    print(f"0x{i.address:x}: {i.mnemonic} {i.op_str}")


print("\n=== Reverse engineering complete ===")

0x0: push rbp
0x1: mov rbp, rsp
0x4: nop 
0x5: pop rbp
0x6: ret 

=== Reverse engineering complete ===
